In [1]:
!pip install GetOldTweets3

In [4]:
from google.colab import drive
import pickle
import GetOldTweets3 as got
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/Colab Notebooks/Internship/Usernames/Mumbai"
users4=open(DATA_PATH+'/slot3.pickle','rb')
real_tweets3=pickle.load(users4)
users4.close()

print(len(real_tweets3))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
672


In [19]:
import multiprocessing 
crawled=set()
m = multiprocessing.Manager()
q = m.Queue()
for i in real_tweets3:      
    q.put(i)
print(q.qsize())
li=list()
def check_if_rich(q):  
  keywords=['singapore','america','spain','germany','france','paris','berlin','europe','bali','indonesia','switzerland','australia','malaysia','united kingdom','london','mauritius','maldives','thailand','dubai','netherland','tokyo','japan','canada','new york','california','fransisco','angeles','vegas','miami','florida','usa','bermuda','iceland','denmark','luxembourg','kuwait','hong kong','greece','mercedes','bmw','lexus','jaguar','royce','fortuner','audi','bentley','porsche','ferrari','volvo','triumph','enfield','interceptor','rover','cadillac','maserati','lamborghini','hilfiger','rolex','rado','tissot','giordano','diesel','fossil','guess','kors','calvin klein','heuer','armani','adidas','nike','piguet','blanc','hublot','patek', 'bandra', 'khar', 'mahalaxmi', 'south bombay', 'worli', 'vashi', 'colaba','dadar','malabar','lokhandwala','andheri','powai','juhu','peddar','altamount','parel', 'trident','marriot','itc','oberoi','taj','hyatt','lalit','sahara','four seasons','westin','orchid','kohinoor','fariyas','marine plaza','meluha','rodas','radisson','shalimar','waterstones','ambassador','ramada','mirage']
  while(q.empty()!=True):
    uname=q.get()
    print(uname)
    if uname not in crawled:
        crawled.add(uname)
        tweetCriteria = got.manager.TweetCriteria().setUsername(uname).setSince("2019-01-01").setUntil("2019-12-31")
        tweets = got.manager.TweetManager.getTweets(tweetCriteria)
        print("Crawling User: ",uname)      
        for j in tweets:
            if any(word in ((j.text).lower()).split() for word in keywords):
                li.append(uname)
                print(li)
                print(j.text)
                break
        print("Going for next user.")
  return li




85


In [0]:
from concurrent.futures import ProcessPoolExecutor
import time
start_time = time.time()

# with concurrent.futures.ProcessPoolExecutor(7) as executor:
#     future = executor.submit(check_if_rich, q)
#     return_value = future.result()



pool = ProcessPoolExecutor(4)
 
future = pool.submit(check_if_rich, q)
result=future.result()

print("--- %s seconds ---" % (time.time() - start_time))



In [21]:
print(result[0])

rohitg1181


In [22]:
import tweepy

consumer_key='rNrnFupaEqKt0eb7hjbdHKdWg'
consumer_secret= 'DTTMoQOrCBmngaXmOnFhrBjdjwtT54x0AbGvNwwuqyYNWwEvc7'
access_token='1002268050513575936-gGrQUmDiMyCxO2Y88lc3ojqNzbtLGm'
access_token_secret='G572YTe2S5TQTTaXhFvl1WyNopa8ilrkgWSlCXBZQwU4C'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

pname=list()
for i in result:
    u=api.get_user(i)
    pname.append(u.name)

print(pname)

['राही', 'ashuleo', 'Shree Mangal Murti', 'Jackal - T Ventures', 'Neelesh Korade', 'Amit Chawla', 'ameya kadam', 'KreateCube', '🌟 Faith', 'Nil Frando', 'Thane Economou', 'merry james', 'Adv.Vivekanand Gupta 🇮🇳', 'KRD Pravin', 'SUNDIP SAXENA', 'Shiksha Jagrit Abhiyaan Foundation', '🏴\u200d☠️🐵🏴\u200d☠️', 'ASHISH SHAH', 'charlotte louise', 'Prthamesh', 'D. V. Bhanushali Real Estate', 'JAGAHA', 'Devendra Date', 'Milind Goraksha', 'Priyankk', 'INFRATECH OFFICIAL', 'Zachy WestSide', 'Bhabtosh', 'LeJagah com - more than a place...', 'DikshaBhoomi Enterprises', 'BTVI Live', 'Runwal Group', 'Rocketsoup', 'Sunil Ramachandani', 'Sonia Kapoor', 'aditya thakur', 'Pritam Biswas', 'Aarefa Johari', 'Varun Bansal', 'Ambi Parameswaran', 'Kapil Yash', 'Ordinary Being', 'QUIDTASK', 'LivNest Real Estate Advisory', 'amit jain', 'Mamura.co', 'Eline Maria', 'Ramesh Ramachandran', '20int 2lugg0']


In [25]:
import string
import re
def clean_tweet(tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

clean=list()

mail=['gmail','yahoo','hotmail','rediff']

for i in pname:
  if (any(word in i.lower() for word in mail))==True:  
      clean.append(i.lower())    
  else:
      demo=deEmojify(i)
      cle=clean_tweet(demo)
      cleaned=cle.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
      clean.append(cleaned)

print(clean)

['', 'ashuleo', 'Shree Mangal Murti', 'Jackal   T Ventures', 'Neelesh Korade', 'Amit Chawla', 'ameya kadam', 'KreateCube', 'Faith', 'Nil Frando', 'Thane Economou', 'merry james', 'Adv Vivekanand Gupta', 'KRD Pravin', 'SUNDIP SAXENA', 'Shiksha Jagrit Abhiyaan Foundation', '', 'ASHISH SHAH', 'charlotte louise', 'Prthamesh', 'D  V  Bhanushali Real Estate', 'JAGAHA', 'Devendra Date', 'Milind Goraksha', 'Priyankk', 'INFRATECH OFFICIAL', 'Zachy WestSide', 'Bhabtosh', 'LeJagah com   more than a place   ', 'DikshaBhoomi Enterprises', 'BTVI Live', 'Runwal Group', 'Rocketsoup', 'Sunil Ramachandani', 'Sonia Kapoor', 'aditya thakur', 'Pritam Biswas', 'Aarefa Johari', 'Varun Bansal', 'Ambi Parameswaran', 'Kapil Yash', 'Ordinary Being', 'QUIDTASK', 'LivNest Real Estate Advisory', 'amit jain', 'Mamura co', 'Eline Maria', 'Ramesh Ramachandran', '20int 2lugg0']


In [26]:
no=['decor','realt','digi','acre','resid','stock','group','article','project','commerc','brick','india','guru','new','market', \
    'enterp','vastu','astro','estate','mall','hous','mumbai','sky','room','home','flat','design','interior','furniture','trend', \
    'pack','build','work','propert','vastu','gym','loan','luxury','bloomberg','business','direct','time',\
    'revenue','day','bank','agen','job','minis','corpo','tech', \
    'boss','nation','associat','developer','broth','invest','people','dna','media','break','bombay','lodha','hiranandani']
finalset=set()
for i in clean:
    if any(word in i.lower() for word in no):
        finalset.add(i)

users=set(clean)
h=users-finalset
print(h)


{'', 'Bhabtosh', 'BTVI Live', 'Thane Economou', 'Jackal   T Ventures', 'Sonia Kapoor', 'Mamura co', 'Zachy WestSide', 'ameya kadam', 'SUNDIP SAXENA', 'Kapil Yash', 'Prthamesh', 'Adv Vivekanand Gupta', 'Rocketsoup', 'Sunil Ramachandani', 'LeJagah com   more than a place   ', 'Faith', 'KreateCube', 'Priyankk', 'Devendra Date', 'KRD Pravin', 'QUIDTASK', 'Neelesh Korade', 'charlotte louise', 'Varun Bansal', 'Eline Maria', 'ashuleo', 'Aarefa Johari', 'Ambi Parameswaran', 'Shiksha Jagrit Abhiyaan Foundation', 'Amit Chawla', 'Shree Mangal Murti', '20int 2lugg0', 'merry james', 'ASHISH SHAH', 'Ramesh Ramachandran', 'Ordinary Being', 'Pritam Biswas', 'Milind Goraksha', 'JAGAHA', 'Nil Frando', 'amit jain', 'aditya thakur'}


In [47]:
for i in users:
  if 'amit' in i:
    print(i)

amit jain


In [0]:
import string
import time
import threading
import urllib
import re
import io
import sys
from time import sleep
import pickle
import pandas as pd


def formats(first, middle, last, domain):
    """
    Create a list of 30 possible email formats combining:
    - First name:          [empty] | Full | Initial |
    - Delimitator:         [empty] |   .  |    _    |    -
    - Last name:           [empty] | Full | Initial |
    """
    list = []
    emails=['gmail','yahoo','rediff','hotmail']

    if any(word in first for word in emails):
        list.append(first)

    elif any(word in middle for word in emails):
        list.append(middle)

    elif any(word in last for word in emails):
        list.append(last)


    elif len(last)==0:
        list.append(first + '@' + domain)                    # first@example.com


    else:
        list.append(first[0] + last + '@' + domain)          # flast@example.com
        list.append(first[0] + '.' + last + '@' + domain)    # f.last@example.com
        list.append(first[0] + '_' + last + '@' + domain)    # f_last@example.com
        list.append(first + '@' + domain)                    # first@example.com
        list.append(first + last + '@' + domain)             # firstlast@example.com
        list.append(first + '.' + last + '@' + domain)       # first.last@example.com
        list.append(first + '_' + last + '@' + domain)       # first_last@example.com
        list.append(first + '-' + last + '@' + domain)       # first-last@example.com

        list.append(first + last[0] + '@' + domain)          # fistl@example.com
        list.append(first + '.' + last[0] + '@' + domain)    # first.l@example.com
        list.append(first + '_' + last[0] + '@' + domain)    # fist_l@example.com
        
        list.append(first[0] + middle + last + '@' + domain)          # fmiddlelast@example.com
        list.append(first[0] + '.' + middle + last + '@' + domain)    # f.middlelast@example.com
        list.append(first[0] + middle + '.' + last + '@' + domain)    # fmiddle.last@example.com
        list.append(first[0] + '_' + middle+ last + '@' + domain)    # f_middlelast@example.com
        list.append(first[0] + middle +'_' + last + '@' + domain)    # fmiddle_last@example.com
        list.append(first + middle+ last + '@' + domain)             # firstmiddlelast@example.com
        list.append(first + middle + '.' + last + '@' + domain)       # firstmiddle.last@example.com
        list.append(first + '.' + middle + last + '@' + domain)       # first.middlelast@example.com
        list.append(first + '_' + middle + last + '@' + domain)       # first_last@example.com
        list.append(first + middle + '_' + last + '@' + domain)       # first_last@example.com
        list.append(first + middle+ last[0] + '@' + domain)          # firstmiddlel@example.com
        list.append(first + '.' + middle +last[0] + '@' + domain)    # first.middlel@example.com
        list.append(first + middle + '.' +last[0] + '@' + domain)    # firstmiddle.l@example.com
        list.append(first + '_' + middle +last[0] + '@' + domain)    # first_middlel@example.com
        list.append(first + middle +'_' + last[0] + '@' + domain)    # firstmiddle_l@example.com        
        
        list.append(last + '@' + domain)                     # last@example.com
        list.append(last + first+ '@' + domain)              # lastfirst@example.com
        list.append(last + '.' + first + '@' + domain)       # last.first@example.com
        list.append(last + '_' + first + '@' + domain)       # last_first@example.com
        list.append(last[0] + '.' + first + '@' + domain)    # l.first@example.com    
        list.append(last[0] + first + '@' + domain)          # lfirst@example.com
        list.append(last + first[0] + '@' + domain)          # lastf@example.com
        list.append(last + '.' + first[0] + '@' + domain)    # last.f@example.com
        list.append(last + '_' + first[0] + '@' + domain)    # last_f@example.com
       
    return(list)


In [40]:
len2=list()
l1=list()
l3=list()
ln=list()

email_list=list()

for i in h:
    try:
        i=i.lower()
        s=i.split()

        if len(s)==2:
            email_list.extend(formats(s[0],'',s[1],'hotmail.com'))
            len2.append(i)
        elif len(s)==1:
            email_list.extend(formats(s[0],'','','hotmail.com'))        
            l1.append(i)
        elif len(s)==3:
            email_list.extend(formats(s[0],s[1],s[2],'hotmail.com'))    
            l3.append(i)
        elif len(s)>3:
            ln.append(i)
            continue    
    except:
        continue        
print(len(email_list))

1094


In [0]:
nino=list()


In [41]:
nino.extend(email_list)

print(len(nino))



6564


In [52]:
emails1=set(nino)
print(len(emails1))


3365
amit.chawla@hotmail.com
amit.chawla@gmail.com
amit.chawla@rediff.com
amit.chawla@yahoo.co.in
amit.chawla@yahoo.com


In [0]:
import pandas as pd

ninoo=list(emails1)
nino1=pd.Series(ninoo)

In [56]:
# DATA_PATH = "/content/drive/My Drive/Colab Notebooks/Internship/Usernames/Thane"
nino1.to_csv("/content/drive/My Drive/Colab Notebooks/Internship/Usernames/Thane/slot1.csv",index=False,encoding='UTF-8')



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
